In [ ]:
import csv
import numpy as np
import pandas as pd
from decision_tree.grow_decision_tree import grow_tree, plot, prune, predict
from decision_tree.open_file import loadCSV
import matplotlib.pyplot as plt
#import networkx as nx

In [ ]:
trainingData3 = loadCSV('../data/iris.csv') # demo data from matlab



decisionTree3 = grow_tree(trainingData3)
plot(decisionTree3)

In [ ]:
def get_hash_dict(d_tree, hash_dict):
    if d_tree.value is not None:
        hash_dict[d_tree.__hash__()] = [d_tree.branch_with_value.__hash__(), d_tree.branch_with_others.__hash__()]
        get_hash_dict(d_tree.branch_with_value,hash_dict)
        get_hash_dict(d_tree.branch_with_others,hash_dict)
def get_neighborhood_list(d_tree):
    dict = {}
    get_hash_dict(d_tree, dict)
    print(dict)
    values = [k for k in dict.keys()]
    for v in dict.values():
        values.extend(v)
    hashes = set(values)
    hash_to_int = {k:i for i,k in enumerate(hashes)}
    print(hash_to_int)
    neighborhood_list = {}
    for k,v in dict.items():
        neighborhood_list[hash_to_int[k]]=[hash_to_int[v[0]], hash_to_int[v[1]]]
    print(neighborhood_list)
    return neighborhood_list

# def draw_graph(nb_list):
#     G = nx.Graph()
#     G.add_nodes_from(nb_list.keys())
#     for k, v in nb_list.items():
#         G.add_edge(k,v[0])
#         G.add_edge(k,v[1])
#     plt.subplot(1)
#     print("piniting...")
#     nx.draw_shell(G, with_labels=True)
print(get_neighborhood_list(decisionTree3))

In [ ]:
trainingData = loadCSV('../notebooks/dane.csv') # demo data from matlab



decisionTree = grow_tree(trainingData)
plot(decisionTree)




In [ ]:
prune(decisionTree3, 0.5, noticication=True) # notify, when a branch is pruned (one time in this example)
plot(decisionTree3)
print('###')
print(predict([6.0, 2.2, 5.0, 1.5], decisionTree3))





# Drzewa decyzyjne

Drzewo decyzyjne to struktura danych zorganizowanych w drzewo, w którym
z każdego węzła (oprócz liści) wychodzą gałęzie reprezentujące poszczególne wartości
atrybutu danych pełniącego funkcję klasyfikatora.

Podstawowym algorytmem budowania drzewa decyzyjnego jest algorytm ID3, bazujący na entropii
informacyjnej i wartości zysku informacyjnego poszczególnych atrybutów.

## Elementy teorii informacji

Przedmiotem teorii informacji jest kwantyfikowanie informacji, czyli ujmowanie jej w sposób
ilościowy, a także detale związane z jej przechowywaniem i przesyłaniem. Poniżej wyjaśnimy podstawowe
dwa pojęcia, na których opiera się konstruowanie drzew decyzyjnych za pomocą algorytmu ID3.

### Entropia

W celu zbudowania drzewa decyzyjnego, będziemy musieli określić zadawane pytania,
a także ustawić je w odpowieniej kolejności. Na każdym etapie drzewa będą znajdowały
się jakieś możliwości, które wyelminowaliśmy, i inne, które czekają na podział. Każde
pytanie dzieli pozostałe możliwości zgodnie z odpowiedzią.

Najlepiej byłoby, gdybyśmy zadawali pytania, które dają odpowiedzi zawierające dużo
informacji na temat tego, co nasze drzewo powinno przewidzieć. "Ilość wnoszonych informacji"
można określić właśnie za pomocą entropii, czyli swoistej miary niepewności związanej z danymi.

Ogólnie rzecz biorąc - dyby wszystkie elementy danego zbioru należały do jednej klasy,
to nie ma żadnej niepewności, a więc entropia byłaby niska. W przypadku równego
rozdziału elementów pomiędzy klasy mamy dużą niepewność, a więc wysoką entropię.

Jeżeli p określa część danych oznaczoną etykietami wskazującymi na przynależność do danej
klasy, to entropię możemy opisać następującym równaniem:

![](https://miro.medium.com/max/1094/1*voRgqFZhfko4ZfG3odSK_Q.png)

Wzór ten zakłada, że 0 log 0 = 0.

Co więcej, przyjmujemy również, że kaźdy czynnik $ -p_{i} log_{2} p_{i} $ jest wartością
nieujemną i bliską zeru dokładnie wtedy, gdy wartość $ -p_{i} $ jest zbliżona do 0
lub 1, co obrazuje poniższy wykres.

![](https://s3-ap-southeast-1.amazonaws.com/he-public-data/Entropydfcb350.jpg)